In [1]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np

In [2]:
# Database-style DataFrame Merges
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],
                'data1': range(7)})
df2 = DataFrame({'key':['a','b','d'],
                'data2': range(3)})

In [3]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [5]:
# MERGE command - many to one merge situtation, merge uses the overlapping column names as the keys
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [6]:
# Here for merging, it's good to specify explicitly
pd.merge(df1,df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
# If the column names are different in each object, we can specify them seperately
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],
                'data1' : range(7)})
df4 = DataFrame({'rkey': ['a','b','d'],
                'data': range(3)})

In [8]:
pd.merge(df3,df4,left_on='lkey', right_on='rkey') # This merging will take inner joint, that's why c and d values aren't present

,data1,lkey,data,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
# To consider outer joint
pd.merge(df1,df2,how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [10]:
# Many-to-Many merges have well defined but not necessarily intuitive behavior
df1 = DataFrame({'key':['b','b','a','c','a','b'],
                'data1': range(6)})
df2 = DataFrame({'key':['a','b','a','b','d'],
                'data2': range(5)})

In [11]:
# Taking only left circke
pd.merge(df1,df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [12]:
# For inner circle
pd.merge(df1,df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [13]:
# Pass a multiple keys, pass a list of column names
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one','two','one'],
                 'lval': [1,2,3]})
right = DataFrame({'key1': ['foo','foo','bar','bar'],
                  'key2': ['one','one','one','two'],
                  'rval': [4,5,6,7]})

In [14]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [15]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [16]:
pd.merge(left,right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [17]:
pd.merge(left,right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [18]:
# merge has a suffixes option for specifying strings to append to overlapping names in the left and right DataFrame objects
pd.merge(left,right,on='key1',suffixes=('_left','_right'))     

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [19]:
# Merging on Index -- Can pass one or both index to be indicate that the index should be used as the merge key
left1 = DataFrame({'key':['a','b','a','a','b','c'],            # only gave DF valued
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5,7]}, index=['a', 'b'])    # gave both DF values and index names

In [20]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [21]:
right1

,group_val
a,3.5
b,7.0


In [22]:
pd.merge(left1,right1, left_on = 'key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [23]:
# Rather than intersect the join keys, can use outer join
pd.merge(left1,right1, left_on='key',right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [24]:
# Hierarchically- indexed data, things are bit more complicated
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                  'key2':[2000,2001,2002,2001,2002],
                  'data':np.arange(5)})
righth = DataFrame(np.arange(12).reshape((6,2)),
                   index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                            [2001,2000,2000,2000,2001,2002]],
                   columns = ['events1','events2'])

In [25]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [26]:
righth

events1  events2
Nevada 2001        0        1
       2000        2        3
Ohio   2000        4        5
       2000        6        7
       2001        8        9
       2002       10       11

In [27]:
pd.merge(lefth,righth, left_on=['key1','key2'], right_index=True)

,data,key1,key2,events1,events2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


In [28]:
pd.merge(lefth,righth, left_on=['key1','key2'], right_index=True, how='outer')

,data,key1,key2,events1,events2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [29]:
# Using the indexes of both sides of the merge is also not an issue
left2 = DataFrame([[1,2], [3,4], [5,6]], index = ['a','c','e'],
                 columns=['Ohio','Nevada'])
right2 = DataFrame([[7,8],[9,10],[11,12],[13,14]],
                  index = ['b','c','d','e'], columns = ['Missiori','Alabama'])

In [30]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [31]:
right2

,Missiori,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [32]:
pd.merge(left2,right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missiori,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [33]:
pd.merge(left2,right2, left_index=True, right_index=True)

,Ohio,Nevada,Missiori,Alabama
c,3,4,9,10
e,5,6,13,14


In [34]:
# Also could written. But it should not be overlapping columns for DF, and it should have same indexes
left2.join(right2, how='outer')            # JOIN function

,Ohio,Nevada,Missiori,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [35]:
# Also JOIN can be used for "left1" and "right1" for that DF
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [36]:
# At last, simple index-on-index merges, can pass list of DF to 'join' function
anotherdf = DataFrame([[7,8],[9,10],[11,12],[16,17]],
                     index=['a','c','e','f'], columns = ['New York','Oregon'])
anotherdf

,New York,Oregon
a,7,8
c,9,10
e,11,12
f,16,17


In [37]:
left2.join([right2,anotherdf])      # can pass multiple DF to join ( default inner joint)

,Ohio,Nevada,Missiori,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9.0,10.0,9,10
e,5,6,13.0,14.0,11,12


In [38]:
left2.join([right2,anotherdf], how='outer')

,Ohio,Nevada,Missiori,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [39]:
# Concatenating Along an Axis - for data combination operations like concatenation, binding or stacking
import numpy as np
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [40]:
np.concatenate([arr, arr], axis = 1)   # by default it will be "0" and take as a columnwise

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [41]:
# "CONCAT" function more explicitly - we have 3 series with no index overlap
s1 = Series([0,1],index = ['a','b'])
s2 = Series([2,3,4], index=['c','d','e'])
s3 = Series([5,6], index=['f','g'])

In [42]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [43]:
pd.concat([s1,s2,s3], axis=1)  # here axis=1, result will be as DF (axis=1 is the columns) This is the outer Union

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [44]:
# Can pass inner Union
s4 = pd.concat([s1 * 5, s3])
pd.concat([s1,s4], axis = 1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [45]:
pd.concat([s1,s4], axis = 1, join = 'inner')

,0,1
a,0,0
b,1,5


In [46]:
# Can also specify the axes to be used on the other axes with 'JOIN_AXES'
pd.concat([s1,s4], axis=1, join_axes = [['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [47]:
# This concatenated pieces are not identifiable in the result. Use 'key' to create a hierarchical index on concatenation axis
result = pd.concat([s1,s1,s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [48]:
# UNSTACK (just brief example)
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [49]:
# In the case of combining column, axis = 1, 'keys' become the DF column header
pd.concat([s1,s2,s3], axis=1, keys = ['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [50]:
df1 = DataFrame(np.arange(6).reshape((3,2)), index = ['a','b','c'],
               columns = ['one','two'])
df2 = DataFrame(5 + np.arange(4).reshape((2,2)), index = ['a','c'],
               columns = ['three','four'])

In [51]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [52]:
df2

,three,four
a,5,6
c,7,8


In [53]:
pd.concat([df1,df2], axis = 1, keys = ['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [54]:
# If we use dict's keys will be used for the "key" option
pd.concat({'level1':df1, 'level2':df2}, axis=1)   # axis = 1 suggests that levels will put at column side

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [55]:
pd.concat([df1,df2], axis = 1, keys = ['level1', 'level2'], names=['upper', 'lower']) # can also define level of column names

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [56]:
# To ignor row index for concat
df1 = DataFrame(np.random.randn(3,4), columns=['a','b','c','d'])
df2 = DataFrame(np.random.randn(2,3), columns=['b','d','a'])

In [57]:
df1

,a,b,c,d
0,-0.567942,0.038655,1.560813,-1.362102
1,0.785491,-1.227096,1.923445,1.255693
2,-0.888875,0.653991,1.557621,-0.758362


In [58]:
df2

,b,d,a
0,-0.063648,1.726232,-0.037035
1,1.224040,-0.150287,1.318387


In [59]:
# we can pass ignore_index=True
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.567942,0.038655,1.560813,-1.362102
1,0.785491,-1.227096,1.923445,1.255693
2,-0.888875,0.653991,1.557621,-0.758362
3,-0.037035,-0.063648,NaN,1.726232
4,1.318387,1.224040,NaN,-0.150287


In [60]:
# Combining data with overlap
a = Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],
          index = ['f','e','d','c','b','a'])
b = Series(np.arange(len(a), dtype=np.float64),
          index = ['f','e','d','c','b','a'])

In [61]:
b[-1] = np.nan

In [62]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [63]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [64]:
# For merging, we can merge a and b ( which are in full index overlapping) by using 'where' function
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [65]:
# By using, "combine_first" method, perform equivalent + data alignment
b[:-2].combine_first(a[2:])   #this means don't take last two values of "b" and first two values of "a"

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [66]:
# Combine_first naturally does the same thing column by column
df1 = DataFrame({'a': [1.,np.nan,5.,np.nan],
                 'b': [np.nan,2.,np.nan,6.],
                 'c': np.arange(2,18,4)})
df2 = DataFrame({'a': [5., 4.,np.nan,3.,7.],
                 'b': [np.nan,3.,4.,6.,8.]})

In [67]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [68]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [69]:
df1.combine_first(df2)      # df2 is joining to df1 so base would be df1 and remaining NaN values will take from df2

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# RESHAPING AND PIVOTING

In [70]:
# for rearranging tabular data, we reffered as 'reshape' or 'pivot' operations
# (1) Reshaping with hierarchical indexing
# Stack - this rotates or pivot from the columns in the data to the rows
# unstack - this pivots from the rows into the columns
data = DataFrame(np.arange(6).reshape((2,3)),
                index = pd.Index(['Ohio','Colorado'], name =  'state'),
                columns = pd.Index(['one','two','three'], name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [71]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [72]:
# For unstacking
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [73]:
# By default the innermost level is unstacked, can unstack a different level by passing a level number
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [74]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [75]:
# Unstacking might introduce missing data if all of the values in the level aren't found in each of the subgroups
s1 = Series([0,1,2,3], index=['a','b','c','d'])
s2 = Series([4,5,6], index = ['c','d','e'])
data2 = pd.concat([s1,s2], keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [76]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [77]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [78]:
# To show NaN values
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [79]:
df = DataFrame({'left':result, 'right': result+5},
              columns=pd.Index(['left','right'], name= 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [80]:
df.unstack('state')     # Unstack state wise

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [81]:
df.unstack('state').stack('side') # Unstack state wise and stack side wise

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

# Data Transformation

# 1) Removing Duplicates

In [82]:
data = DataFrame({'k1': ['one'] *3 + ['two'] * 4,
                 'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [83]:
# To find duplicated returns a boolean series indicating whether each row is duplicate or not
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [84]:
# "drop_duplicates" returns a DF where the duplicated array is TRUE
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [85]:
# Suppose if we want to duplicates only based on the 'k1' column
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [86]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


# Transforming Data using a function or Mapping


In [88]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami','corned beef', 'Bacon',
                           'pastrami','honey ham','nova lox'],
                 'ounces': [4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [89]:
# Suppose we wanted to add type of animal that each food came from
meat_to_animal = {'bacon':'pig', 'pulled pork': 'pig','pastrami':'cow','corned beef':'cow',
                 'honey ham':'pig','nova lox':'salmon'}
meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

In [90]:
# Here for mapping, some meats above are capatalized and others are not. we need to convert into lowercase
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)  # create a column in data with mapping lower string
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [91]:
# can also have passed function that does all the work
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

# Replacing Values



In [92]:
# map can modify a subset of values in object,"replace" provides a simpler and more flexible way to do so
data = Series([1.,-999.,2.,-999.,-1000.,3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [93]:
# To replace this value with NA, use 'replace' to producing a new Series
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [94]:
# If we want to replace multiple values at once, pass list instead the substitute value
data.replace([-999,-1000],np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [95]:
# Can place different replacement for each values, pass a list of substitutes
data.replace([-999,-1000], [np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [96]:
# Argument passed can also be a dict
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

# Renaming Axis Indexes

In [97]:
data = DataFrame(np.arange(12).reshape((3,4)),
                index = ['Ohio','Colorado','New York'],
                columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [98]:
data.index.map(str.upper)     # Like Series, the axis indexes have a "map" method (here, converted into Upper Case)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [99]:
# converted index (UPPER CASE) into "data" file
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [100]:
# Create a transformed version of a data set without modifying the original, use "rename"
data.rename(index = str.title, columns=str.upper)    # In index (first letter capital, for columns all letter capital)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [101]:
# With the use of "rename" can also providing new values for a subset of the axis labels
data.rename(index={'OHIO':'INDIANA'},
           columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [102]:
data   # our original file

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [103]:
# If to make a copy of DataFrame, if wish to chage data set permantently use "inplace=True"
_ = data.rename(index={'OHIO':'INDIANA'}, inplace=True)

In [104]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


# Discretiation and Binning

In [105]:
# Suppose we want to discretized or otherwised separated into 'bins' for analysis, here we are using age buckets
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

In [106]:
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)

In [107]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [108]:
# For their labels
cats.labels

/home/vivek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  


array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [109]:
# for their levels
cats.levels

AttributeError: 'Categorical' object has no attribute 'levels'

In [110]:
# For how many values in each bins
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [111]:
# on above, we can change that parenthesis and square bracket and can switch it
pd.cut(ages, [18,26,36,61,100], right=False)     # right = False means, it won't take by default square bracket

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [112]:
# Can also pass a label name for each bin
group_name = ['Youth','YoungAdult','MiddleAge','Senior']
pd.cut(ages,bins, labels=group_name)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAge, MiddleAge, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAge < Senior]

In [113]:
# If we CUT a integer number of bins instead of explicit bin edges, it will compute equal-length bins 
data = np.random.randn(20)
pd.cut(data,4,precision=2)

[(0.61, 1.55], (0.61, 1.55], (0.61, 1.55], (-0.33, 0.61], (-2.21, -1.27], ..., (-0.33, 0.61], (-0.33, 0.61], (0.61, 1.55], (-1.27, -0.33], (-0.33, 0.61]]
Length: 20
Categories (4, interval[float64]): [(-2.21, -1.27] < (-1.27, -0.33] < (-0.33, 0.61] < (0.61, 1.55]]

In [114]:
# "qcut" will cut into sample quantiles, will cut roughly in equal-size bin
data = np.random.randn(1000)    # Normally distributed
cats = pd.qcut(data, 4)  # cut into quartiles
cats

[(-0.62, 0.037], (0.698, 4.002], (-0.62, 0.037], (0.037, 0.698], (0.698, 4.002], ..., (-2.691, -0.62], (0.698, 4.002], (0.037, 0.698], (-2.691, -0.62], (0.698, 4.002]]
Length: 1000
Categories (4, interval[float64]): [(-2.691, -0.62] < (-0.62, 0.037] < (0.037, 0.698] < (0.698, 4.002]]

In [115]:
pd.value_counts(cats)

(0.698, 4.002]     250
(0.037, 0.698]     250
(-0.62, 0.037]     250
(-2.691, -0.62]    250
dtype: int64

In [116]:
# Same as above, can pass own quantiles between 0 and 1
a = pd.qcut(data, [0,0.1,0.5,0.9,1.])
a

[(-1.246, 0.037], (1.34, 4.002], (-1.246, 0.037], (0.037, 1.34], (0.037, 1.34], ..., (-1.246, 0.037], (0.037, 1.34], (0.037, 1.34], (-1.246, 0.037], (1.34, 4.002]]
Length: 1000
Categories (4, interval[float64]): [(-2.691, -1.246] < (-1.246, 0.037] < (0.037, 1.34] < (1.34, 4.002]]

# Detecting and Filtering Outliers

In [117]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [118]:
# Suppose we wanted to find values in one of the columns exceeding 'three' magnitude
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [119]:
# Find values above 3 in every column. (There is one or more value above 3 in each row), Use 'ANY' methos
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [120]:
# Values can just as easily be set based on these criteria. below code to cap values outside the internal -3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3         # 'np.sign' returns an array of 1 and -1 depending on the sign of the values
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


# Permutation and Random Sampling

In [121]:
# Permuting (randomly recording) a series or row in a DataFrame is to do using the 'numpy.random.permutation'
df = DataFrame(np.arange(20).reshape(5,4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [122]:
sampler = np.random.permutation(5)    # permutation will re-order the index, and it should be same or below available indices
sampler

array([1, 0, 2, 3, 4])

In [123]:
df.take(sampler)   # Array can then be used in 'ix' based indexing or the 'take' function

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
